In [1]:
from threeDprediction import *
from threeDprediction_multi import *
from pore_analysis import *
from visualization import *
import os
import numpy as np
from skimage import io

# Welcome to the Jupyter Notebook for the utilization of UTILE-Pore

With this notebook, you will be guided through the code and make it possible for you to analyze your porous materials tomographies automatically.

This notebook should help you to run the automated segmentation of the volumes in your images and afterward to apply the diverse functions to extract the information of interest from your data and visualize the results.

Already integrated functions are:

- GDL/porous material
- Pore size distribution analysis of the pores with PoreSpy
- Surface roughness calculation
- Tortuosity simulation with PoreSpy
- Permeability estimation using Kozeny-Carman equation
- Calculation of the solid surface ratio


For MPL
- Layer thickness calculation
- MPL crack analsis (ratio, size distribution, etc.)
- MPL heatmap for thickness variation
- MPL intrusion measurement
- MPL and GDL contact surface calculation

In [2]:
#Give a name to your project

case_name = "example1"

#First, we need to specify the folder where the tomograph slices (or tif stack) are stored

image_path= "C:/Users/a.colliard/Desktop/aimys_project/CT_crops/new/totake/39bb2.tif"

#Secondly, we need to specify where is your model stored

model_path = "C:/Users/a.colliard/Downloads/HRNET_HRNET_multi_fusev2_dataset_noaug_-0.6808-23.keras"

#It is also required to create a folder to store the predicted masks
os.makedirs(f"./{case_name}/", exist_ok=True)

# We set the path to the csv file to store all the extracted data
csv_file = f'./{case_name}/csv_test.csv'

In [ ]:
# Now we start with the segmentation of the volume
# If you just have a porous materials and need a binary segmentation, run the following lines

# input_volume = io.imread('/p/project1/claimd/Andre/Aimy/Dataset/cal_fusev2/test/Toray1202.tif')
# predicted_volume = process_and_predict(image_path, model_path)
# io.imsave(f'./predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

#If you are analyzing a GDL with MPL, then you can run a multiclass semantic segmentation with the following lines:
input_volume = io.imread(image_path)
predicted_volume = process_and_predict_multi(input_volume, model_path)
io.imsave(f'./{case_name}/predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


[11:26:41] WARNING  Mixed precision compatibility check (mixed_float16): WARNING  ]8;id=548713;file://C:\Users\a.colliard\Miniconda3\envs\pore_test\lib\site-packages\keras\mixed_precision\device_compatibility_check.py\device_compatibility_check.py]8;;\:]8;id=661095;file://C:\Users\a.colliard\Miniconda3\envs\pore_test\lib\site-packages\keras\mixed_precision\device_compatibility_check.py#109\109]8;;\
                    The dtype policy mixed_float16 may run slowly because this                                     
                    machine does not have a GPU. Only Nvidia GPUs with compute                                     
                    capability of at least 7.0 run quickly with mixed_float16.                                     
                    If you will use compatible GPU(s) not attached to this host,                                   
                    e.g. by running a multi-worker model, you can ignore this                                      
                    warning. This message will only be logged once                                                 

In [3]:
# Now, lets visualize the segmented volume
predicted_volume = io.imread(f'./{case_name}/predicted_{case_name}.tif')
visualize_volume(predicted_volume, case_name, False)

In [4]:
# We can directly start wiht the property extraction of the porous material or GDL
# We start with the pore size distribution analysis
porosity, results, avg_pore, sd = calculate_psd(predicted_volume, csv_file, case_name, voxel_size=5)

Porosity: 0.6744162305979567


  0%|          | 0/25 [00:00<?, ?it/s]

Sizes shape: (46, 1803, 1035)
Results: ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Results of pore_size_distribution generated at Tue Sep 17 13:49:42 2024
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
LogR                      Array of size (10,)
pdf                       Array of size (10,)
cdf                       Array of size (10,)
satn                      Array of size (10,)
bin_centers               Array of size (10,)
bin_edges                 Array of size (11,)
bin_widths                Array of size (10,)
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Average Pore Size (microns): 4.300724564067334
Standard Deviation (microns): 2.9284256014513987


In [5]:
#Tortuosity simulation (for non-optimized devices can take several hours)
tortuosity = 1.85
#tortuosity = tortuosity_simulation(predicted_volume, csv_file)

In [6]:
# Having the above mentioned properties, we can estimate the permeability
# Estimate permeability
ssa = calculate_ssa(predicted_volume)
permeability = estimate_permeability(porosity, csv_file, ssa, tortuosity)
print(f'Permeability: {permeability}')

Permeability: 1.6617189993407624


C:\Users\a.colliard\Documents\GitHub\UTILE-pore\UTILE-pore\pore_analysis.py:183: RuntimeWarning: overflow encountered in long_scalars
  solid_volume = np.sum(binary_volume) * (voxel_size ** 3)


In [7]:
# Then surface roughness calculation of the material
#Calculate surface roughness
Ra, Rq = calculate_surface_roughness_from_surface(predicted_volume)
with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write data
        writer.writerow(['##### Surface roughness #####'])
        writer.writerow(['Arithmetic_Mean_Roughness_(Ra)', Ra])
        writer.writerow(['Root_Mean_Square_Roughness_(Rq)', Rq])
print(f"Arithmetic Mean Roughness (Ra): {Ra}")
print(f"Root Mean Square Roughness (Rq): {Rq}")


[1.110223e-16 1.000000e+00 1.000000e+00 ... 9.000000e+00 9.000000e+00
 8.000000e+00]
Arithmetic Mean Roughness (Ra): 11.688429117202759
Root Mean Square Roughness (Rq): 13.575758934020996


In [8]:
#Calculacte solid surface ratio
calculate_solid_surface_ratio(predicted_volume, csv_file, gdl=1, side='bottom')

(1803, 1035)
Solid surface ratio: 0.23308763440428057


0.23308763440428057

In [9]:
# Now we can move to the functions specialized for MPL containing GDLs
# We can caluclate the MPL and GDL thickness
MPL_GDL_thickness(predicted_volume, csv_file, axis=0, mpl=2, gdl=1)


MPL avg Thickness 6
GDL avg Thickness 8
thickness_along_z (GDL)  [109240 160660 215257 273947 337949 385268 417800 444269 471639 493537
 510132 511173 496178 471539 441101 404341 368758 333470 297015 266439
 240807 218165 198257 181018 165908 152374 143074 141879 149776 165416
 187752 216425 250535 291141 330410 367165 403409 437987 465788 482341
 491349 491912 486257 466566 443273 434966]
Max GDL Thickness 230 Max MPL Thickness 155


(155,
 array([[ 0,  6,  7, ...,  4,  1,  0],
        [ 4,  5,  7, ...,  5,  3,  2],
        [ 4,  5,  8, ...,  6,  5,  4],
        ...,
        [ 7,  7,  8, ...,  6,  4,  3],
        [ 7,  7, 10, ...,  4,  5,  6],
        [ 7,  8, 11, ...,  6,  8,  7]]))

In [10]:
#Calculate MPL crack analysis
crack_ratio, crack_count, crack_labels, crack_sizes, slice_image = MPL_crack_analysis(predicted_volume,case_name, csv_file)
print(f"Crack Ratio: {crack_ratio}")
print(f"Crack Count: {crack_count}")
#print(f"Crack Sizes: {crack_sizes}")
plot_crack_labels(slice_image, case_name, crack_labels)


Crack Ratio: 0.2534289335273203
Crack Count: 486


In [11]:
# Example usage:
Ra, Rq, Ra_std_dev, Ra_CoV, avg_thickness = MPL_intrusion_roughness(predicted_volume, csv_file, mpl=2, voxel_size=5)
print(f"Global Roughness Ra: {Ra}, Rq: {Rq}")
print(f"Standard Deviation of Local Ra: {Ra_std_dev}")
print(f"Coefficient of Variation of Local Ra: {Ra_CoV}")
print(f"Average Thickness: {avg_thickness}")

mpl Thickness 6
[0. 0. 0. ... 3. 3. 3.]
Global Roughness Ra: 4.787220358848572, Rq: 5.517091155052185
Standard Deviation of Local Ra: 0.49059549415770676
Coefficient of Variation of Local Ra: 0.102480240595339
Average Thickness: 6


In [12]:
# MPL heatmap creation
MPL_heatmap(predicted_volume,case_name, 2)

In [13]:
# Calculate fiber touching MPL voxels
MPL_count_touching_voxels(predicted_volume, csv_file, mpl_class=1, fiber_class=2)

MPL voxels touching GDL:  880834


880834